In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lensk,60.73,114.93,74.48,53,100,2.08,RU,1593607131
1,1,jamestown,42.10,-79.24,69.01,88,75,6.93,US,1593606990
2,2,omboue,-1.57,9.26,77.65,73,100,10.65,GA,1593607131
3,3,pyapon,16.28,95.68,80.02,84,100,11.86,MM,1593607131
4,4,port alfred,-33.59,26.89,80.01,28,0,5.99,ZA,1593607132


In [3]:
# Configure gmaps
 

gmaps.configure(api_key=g_key)
 

In [4]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
81,81,cape town,-33.93,18.42,71.01,60,0,1.12,ZA,1593606891
85,85,east london,-33.02,27.91,73.40,43,0,4.70,ZA,1593607146
144,144,coruripe,-10.13,-36.18,79.29,65,0,5.66,BR,1593607156
150,150,piacabucu,-10.41,-36.43,79.66,61,0,5.99,BR,1593607157
155,155,kruisfontein,-34.00,24.73,72.72,31,0,9.42,ZA,1593607157
248,248,hwange,-18.37,26.50,79.07,27,0,5.77,ZW,1593607172
272,272,sao jose da coroa grande,-8.90,-35.15,79.90,66,0,8.32,BR,1593607176
281,281,vidim,56.41,103.11,75.92,55,0,4.21,RU,1593607177
316,316,upata,8.01,-62.40,79.00,85,0,1.99,VE,1593607183
364,364,amuntai,-2.42,115.25,75.85,93,0,0.83,ID,1593607190


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
81,cape town,ZA,-33.93,18.42,
85,east london,ZA,-33.02,27.91,
144,coruripe,BR,-10.13,-36.18,
150,piacabucu,BR,-10.41,-36.43,
155,kruisfontein,ZA,-34.00,24.73,
248,hwange,ZW,-18.37,26.50,
272,sao jose da coroa grande,BR,-8.90,-35.15,
281,vidim,RU,56.41,103.11,
316,upata,VE,8.01,-62.40,
364,amuntai,ID,-2.42,115.25,


In [7]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
81,cape town,ZA,-33.93,18.42,Southern Sun Waterfront Cape Town
85,east london,ZA,-33.02,27.91,Tu Casa
144,coruripe,BR,-10.13,-36.18,Pousada e Hotel Litoral Sul
150,piacabucu,BR,-10.41,-36.43,Piaçabuçu News
155,kruisfontein,ZA,-34.00,24.73,Oyster Bay House Rental
248,hwange,ZW,-18.37,26.50,Hwange Colliery Guest House
272,sao jose da coroa grande,BR,-8.90,-35.15,Crôa Mares Hotel
281,vidim,RU,56.41,103.11,
316,upata,VE,8.01,-62.40,Posada Villa Nela
364,amuntai,ID,-2.42,115.25,Hotel Balqis


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))